In [1]:
# Develop a python application that uses Kalman filter for smoothing the output of the mouse pointer. 
# Define process model, transition matrix and system noise. Write code as python application with comments.
# Required submission: code of the system, recorded avi file of the system. Use OpenCV
# VideoWriter class for storing the video

In [2]:
from numpy.linalg import inv
import numpy as np

In [3]:
class KalmanFilter:
    def __init__(self, x, P, F, R, Q, H, m):
        self.x = x
        self.P = P
        self.F = F
        self.R = R
        self.Q = Q
        self.H = H
        self.m = m
    
    def predict(self):
        self.x = np.dot(self.F, self.x)
        self.P = np.dot(self.F, self.P).dot(self.F.T) + self.Q
        return self.x

    def correct(self, m):
        S = self.H.dot(self.P).dot(self.H.T) + self.R
        K = np.dot(self.P, self.H.T).dot(inv(S))
        self.x += K.dot(m - self.H.dot(self.x))
        self.P = self.P - np.dot(K, self.H).dot(self.P)


In [4]:
# state = [0,0]
# covariance_matrix = [[1,0], 
#                      [0,1]]

# x = [x, y, v_x, v_y]
x = np.array([0,0,0,0])
m = np.array([0,0,0,0])

# P covariance matrix
P = np.array([[5, 0, 0, 0],
             [0, 5, 0, 0],
             [0, 0, 5, 0],
             [0, 0, 0, 5]])
# F - state_transition_matrix
F = np.array([[1, 0, 0.2, 0],
             [0, 1, 0, 0.2],
             [0, 0, 1, 0  ],
             [0, 0, 0, 1  ]])

# R - noise matrix
R = np.array([[0.1, 0, 0, 0],
             [0, 0.1, 0, 0],
             [0, 0, 0.1, 0],
             [0, 0, 0, 0.1]])

# Q noise co-variance matrix
Q = np.array([[0, 0, 0, 0],
             [0, 0, 0, 0],
             [0, 0, 0.1, 0],
             [0, 0, 0, 0.1]])

# H measurement matrix
H = np.array([[1, 0, 1, 0],
             [0, 1, 0, 1],
             [0, 0, 0, 0],
             [0, 0, 0, 0]])

In [5]:
from tkinter import *
from collections import deque  

kf = KalmanFilter(x, P, F, R, Q, H, m)

window=Tk()
canvas = Canvas(window,  width=900, height=600)
canvas.pack()

# canvas.create_oval(x-r,y-r,x+r,y+r)

# window.title('Kalman Filter Demo')
# window.geometry("900x600+300+200")
r = 5

# mouth_path = deque([]) 
mouth_path = [(0,0)]
lines = deque([]) 
def motion(event):
    x, y = event.x, event.y
    predicted_x, predicted_y = kf.predict()[:2]
    prev_x, prev_y = mouth_path[-1]
    mouth_path.append((predicted_x, predicted_y))
    new_line = canvas.create_line(prev_x, prev_y, predicted_x, predicted_y, 
                                  fill='green', width=3)
    lines.append(new_line)
    if len(lines) > 200:
        to_del = lines.popleft()
        canvas.delete(to_del)
    kf.correct(np.array([x, y, 0, 0]))
    return

window.bind('<Motion>', motion)
window.mainloop()